# ChatXAI

本页面将帮助您开始使用 xAI [聊天模型](../../concepts/chat_models.mdx)。有关 `ChatXAI` 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/xai/chat_models/langchain_xai.chat_models.ChatXAI.html)。

[xAI](https://console.x.ai/) 提供了一个 API 来与 Grok 模型进行交互。

## 概述
### 集成详情

| 类 | 包 | 本地 | 可序列化 | [JS 支持](https://js.langchain.com/docs/integrations/chat/xai) | 包下载量 | 包最新版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatXAI](https://python.langchain.com/api_reference/xai/chat_models/langchain_xai.chat_models.ChatXAI.html) | [langchain-xai](https://python.langchain.com/api_reference/xai/index.html) | ❌ | beta | ✅ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-xai?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-xai?style=flat-square&label=%20) |

### 模型功能
| [工具调用](../../how_to/tool_calling.ipynb) | [结构化输出](../../how_to/structured_output.ipynb) | JSON 模式 | [图像输入](../../how_to/multimodal_inputs.ipynb) | 音频输入 | 视频输入 | [Token 级流式输出](../../how_to/chat_streaming.ipynb) | 原生异步 | [Token 使用量](../../how_to/chat_token_usage_tracking.ipynb) | [Logprobs](../../how_to/logprobs.ipynb) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ❌ | ❌ | ❌ | ❌ | ✅ | ❌ | ✅ | ✅ |

## 设置

要访问 xAI 模型，您需要创建一个 xAI 账户，获取 API 密钥，并安装 `langchain-xai` 集成包。

### 凭证

前往 [此页面](https://console.x.ai/) 注册 xAI 并生成 API 密钥。完成此操作后，设置 `XAI_API_KEY` 环境变量：

In [ ]:
import getpass
import os

if "XAI_API_KEY" not in os.environ:
    os.environ["XAI_API_KEY"] = getpass.getpass("Enter your xAI API key: ")

为了启用模型调用的自动化跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [2]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

LangChain xAI 集成存在于 `langchain-xai` 包中：

In [3]:
%pip install -qU langchain-xai

## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全：

In [4]:
from langchain_xai import ChatXAI

llm = ChatXAI(
    model="grok-beta",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## 调用

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 30, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'grok-beta', 'system_fingerprint': 'fp_14b89b2dfc', 'finish_reason': 'stop', 'logprobs': None}, id='run-adffb7a3-e48a-4f52-b694-340d85abe5c3-0', usage_metadata={'input_tokens': 30, 'output_tokens': 6, 'total_tokens': 36, 'input_token_details': {}, 'output_token_details': {}})

In [6]:
print(ai_msg.content)

J'adore programmer.


## 链接

我们可以像这样将我们的模型与提示模板[链接](../../how_to/sequence.ipynb)：

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 25, 'total_tokens': 32, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'grok-beta', 'system_fingerprint': 'fp_14b89b2dfc', 'finish_reason': 'stop', 'logprobs': None}, id='run-569fc8dc-101b-4e6d-864e-d4fa80df2b63-0', usage_metadata={'input_tokens': 25, 'output_tokens': 7, 'total_tokens': 32, 'input_token_details': {}, 'output_token_details': {}})

## 工具调用

ChatXAI 拥有一个 [工具调用](https://docs.x.ai/docs#capabilities) (此处我们交替使用“工具调用”和“函数调用”) API，该 API 允许您描述工具及其参数，并让模型返回一个 JSON 对象，其中包含要调用的工具和该工具的输入。工具调用对于构建使用工具的链和代理，以及从模型获取更通用的结构化输出来说极其有用。

### ChatXAI.bind_tools()

通过 `ChatXAI.bind_tools`，我们可以轻松地将 Pydantic 类、字典模式、LangChain 工具，甚至是函数作为工具传递给模型。在底层，这些会被转换为 OpenAI 工具模式，如下所示：
```
{
    "name": "...",
    "description": "...",
    "parameters": {...}  # JSONSchema
}
```
并在每次模型调用时传递。

In [8]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm_with_tools = llm.bind_tools([GetWeather])

In [9]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in San Francisco",
)
ai_msg

AIMessage(content='I am retrieving the current weather for San Francisco.', additional_kwargs={'tool_calls': [{'id': '0', 'function': {'arguments': '{"location":"San Francisco, CA"}', 'name': 'GetWeather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 151, 'total_tokens': 162, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'grok-beta', 'system_fingerprint': 'fp_14b89b2dfc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-73707da7-afec-4a52-bee1-a176b0ab8585-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'San Francisco, CA'}, 'id': '0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 11, 'total_tokens': 162, 'input_token_details': {}, 'output_token_details': {}})

## 实时搜索

xAI 支持一项[实时搜索](https://docs.x.ai/docs/guides/live-search)功能，该功能使 Grok 能够利用网络搜索结果来支持其答案：

In [ ]:
from langchain_xai import ChatXAI

llm = ChatXAI(
    model="grok-3-latest",
    search_parameters={
        "mode": "auto",
        # Example optional parameters below:
        "max_search_results": 3,
        "from_date": "2025-05-26",
        "to_date": "2025-05-27",
    },
)

llm.invoke("Provide me a digest of world news in the last 24 hours.")

请参阅 [xAI 文档](https://docs.x.ai/docs/guides/live-search) 以了解完整的网页搜索选项。

## API 参考

如需了解 `ChatXAI` 所有特性和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/xai/chat_models/langchain_xai.chat_models.ChatXAI.html)。